In [1]:
import numpy as np
import pandas as pd
from pymongo import MongoClient
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import pingouin as pg
import tkinter as tk
from tkinter import filedialog
from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg, NavigationToolbar2Tk)
from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure
import traceback
from tkinter import messagebox
from PIL import ImageTk, Image


## Database and data import classes

In [2]:
class Database:
    DATABASE = None
    
    #predefined methods for accessing the PyMongo database for efficiency in code reuse.
    #static methods used as there will only be one instance of the database

    @staticmethod
    def initialise():
        client = MongoClient()
        Database.DATABASE = client.database
    
    @staticmethod
    def insert(collection, data):
        Database.DATABASE[collection].insert_many(data)
        
    @staticmethod
    def insert_one(collection, data):
        Database.DATABASE[collection].insert_one(data)    
        
    @staticmethod
    def find(collection, query, *presentation):
        return Database.DATABASE[collection].find(query, *presentation)
        
    @staticmethod
    def delete_collection(collection):
        Database.DATABASE[collection].drop()
    
    @staticmethod
    def update(collection, query, update):
        Database.DATABASE[collection].update_many(query, update)
        
    @staticmethod
    def delete_duplicates(collection, type_field, identity_field):
        identity_field = "$"+identity_field
        cursor = Database.DATABASE[collection].aggregate([
            {'$match': {type_field: {'$exists': True}}},
            {"$group": {"_id": identity_field, "unique_ids": {"$addToSet": "$_id"}, "count": {"$sum": 1}}},
            {"$match": {"count": { "$gte": 2 }}}
        ], allowDiskUse = True)
        
        response = []
        for doc in cursor:
            del doc["unique_ids"][0]
            for id in doc["unique_ids"]:
                response.append(id)
        
        x = Database.DATABASE[collection].delete_many({"_id": {"$in": response}})
        return x.deleted_count
    
    def delete(collection, _ids):
        Database.DATABASE[collection].delete_many()
        
    def type_dict(type_field):
        # fields unique to each document type are used to select only those documents in searching using pymongo $exists
        # this function translates docment name to unique field and returns that field
        type_dict = {'Inventory': 'OWNER ZIP', 'Inspections': 'GRADE', 'Violations': 'VIOLATION CODE', 'Seats': 'seats', 'Program Status': 'type'}
        type_var = type_dict[type_field]
        return type_var
        
    def identity_dict(identity_field):
        # these are the field used as unique reference in each document type
        identity_dict = {'Inventory': 'FACILITY ID', 'Inspections': 'SERIAL NUMBER', 'Violations': 'bespoke_id'}
        identity_var = identity_dict[identity_field]
        return identity_var
    
    def count(collection, type_field):
        type_var = Database.type_dict(type_field)
        Database.initialise()
        x = Database.DATABASE[collection].count_documents({type_var: {'$exists': True}})
        return x
        
    

In [3]:
class Importer:
      
    def generic_importer(self, path):
        
        #read data to pandas DataFrame
        self.import_data = pd.read_csv(path)
        self.import_data.reset_index()
        
        # create a unique reference for each violation in order to be able to identify duplicates
        if 'POINTS' in self.import_data.columns:
            self.import_data['bespoke_id'] = self.import_data['SERIAL NUMBER'] + self.import_data['VIOLATION CODE']
            
        # bulk insert to pymongo, using Pandas to dict method
        Database.insert('FACILITIES', self.import_data.to_dict('records'))
        return "upload complete"

## Data cleaning functions used in 'clean my data' screen

In [4]:
class Cleaning:
    
    # all methods called by clean screen except the latter, which is used for data preparation in analysis tasks
    def delete_duplicates(self, type_field, identity_field):
        x = Database.delete_duplicates('FACILITIES', type_field, identity_field)
        return x
    
    def get_seating_info(self):
        # creates seating data from the PE DESCRIPTION field in line with client requirements
        # this is stored in its own documents, updating existing documents having been found to be very slow
        
        facility_pedesc = pd.DataFrame(Database.find('FACILITIES', {'OWNER STATE': {'$exists': True}}, {'_id': 0, 'FACILITY ID': 1, 'PE DESCRIPTION': 1,}))
        facility_pedesc['seats'] = facility_pedesc['PE DESCRIPTION'].apply(lambda s: s[s.find("(")+1:s.find(")")])
        facility_pedesc['type'] = 'seats'
        facility_pedesc.drop('PE DESCRIPTION', axis = 1, inplace = True)
        Database.insert('FACILITIES', facility_pedesc.to_dict('records'))
        return 'seating data extracted and added to the data'

    def insert_prog_status(self):
        # creates a document for every facility with a program status, based on the status at the most recent inspection
        prog_status = pd.DataFrame(Database.find('FACILITIES', {'PROGRAM STATUS':{'$exists': True}}, {'_id': 0, 'ACTIVITY DATE': 1, 'FACILITY ID': 1, 'PROGRAM STATUS': 1}))
        last_status = prog_status.groupby('FACILITY ID', as_index=False).last()
        last_status['type'] = 'status'
        last_status.drop('ACTIVITY DATE', axis = 1, inplace = True)
        last_status['FACILITY ID'].drop_duplicates(inplace = True)
        last_status['FACILITY ID'].dropna(inplace = True)
        Database.insert('FACILITIES', last_status.to_dict('records'))
        return "Program status by facility added to database"
    
    def remove_inactive(self, data):
        # takes a dataframe, merges with program status document from the database, drops inactive, returns dataframe 
        prog_stat = pd.DataFrame(Database.find('FACILITIES', {'type': 'status'}, {'_id': 0, 'FACILITY ID': 1, 'PROGRAM STATUS': 1}))
        merged_data = pd.merge(prog_stat, data, on = 'FACILITY ID')
        return_data = merged_data.drop(merged_data[merged_data['PROGRAM STATUS'] == 'INACTIVE'].index)
        return return_data

## Data cleaning and wrangling for visualisations

In [5]:
class ViolationsData:
                          
    def get_violations_graph_data(self):
                            
        # from violation documents: find violation codes, with serial number identifying the inspection each was part of
        violations = pd.DataFrame(Database().DATABASE['FACILITIES']\
            .find({'VIOLATION CODE': {'$exists': True}}, {'_id': 0, 'SERIAL NUMBER': 1, 'VIOLATION CODE': 1}))

        # groupby serial number then violation code
        # aggreate violation code to provide a count of of each violation type by serial number
        violations_grouped = violations.groupby(['SERIAL NUMBER', 'VIOLATION CODE']).agg({'VIOLATION CODE': 'count'})

        # sparse matrix of serial numbers and violation codes
        violation_count_unstack = violations_grouped['VIOLATION CODE'].unstack()
        violation_count_unstack.fillna(0, inplace = True)

        # from inspection documents: find facility id and serial number
        facilityid_serialno = pd.DataFrame(Database.DATABASE['FACILITIES'].find({'ACTIVITY DATE':{'$exists': True}},\
                                                                  {'_id': 0, 'FACILITY ID': 1, 'SERIAL NUMBER': 1}))

        fac_serial_active = Cleaning().remove_inactive(facilityid_serialno)
        
        # merge the two dataframes on serial number and group by facility ID to provide a facility rather than inspection level
        #   matrix

        violations_facilities = pd.merge(violation_count_unstack, fac_serial_active, on = 'SERIAL NUMBER')\
                .groupby('FACILITY ID').sum().T

        # sum number of non zero fields - provides count of facilities that have commited each violation
        violations_facilities['sum'] = (violations_facilities != 0).sum(axis = 1)
        violations_facilities = violations_facilities['sum']
        vio_graph_data = violations_facilities.sort_values(ascending = False)

        return vio_graph_data
 


In [6]:
class MeanMedModeData:
    def __init__(self):
        self.cleaned_base_data = None
        self.return_data = None 
        
    def _get_clean_data(self, group_variable):
        # called by the below function. Gets data from the database, cleans and manipulates 
        base_data = pd.DataFrame(Database.DATABASE['FACILITIES'].find({'EMPLOYEE ID': {'$exists': True}},\
                                        {'_id': 0, 'FACILITY ID': 1, 'SCORE': 1, group_variable: 1, 'ACTIVITY DATE': 1}))
        
        # get year using datetime to allow grouping in line with client requirement
        base_data['year'] = pd.DatetimeIndex(base_data['ACTIVITY DATE']).year
        
        # actions specific to each grouping variable - zip codes stripped to two digits
        # seat information imported from database and merged in
        if group_variable == 'Zip Codes':
            base_data['Zip Codes'] = base_data['Zip Codes'].apply(lambda x: str(x)[0:2])
        if group_variable == 'seats':
            seats_data = pd.DataFrame(Database.DATABASE['FACILITIES'].find({'seats': {'$exists': True}}, {'_id': 0,}))
            base_data = base_data.merge(seats_data, on = 'FACILITY ID')
        
        # cleaning to remove na values, including zero scores and bespoke'na', remove inactive facilities
        base_data.dropna(subset = [group_variable, 'SCORE'], inplace = True)    
        base_data.drop(base_data[base_data[group_variable] == 0].index, inplace = True)
        base_data.drop(base_data[base_data[group_variable] == 'na'].index, inplace = True)
        self.cleaned_base_data = Cleaning().remove_inactive(base_data)
    
    
    def wrangle_mean_med_data(self, group_variable):
        # wrangles data to prepare for visualisation and returns it
        
        self._get_clean_data(group_variable)
        self.return_data = self.cleaned_base_data.groupby(['year', group_variable]).mean().round(1)
        
        #get required measures of central tendency using native pandas functions
        self.return_data['median'] = self.return_data.groupby(['year', group_variable])['SCORE'].median()
        self.return_data['mode'] = self.return_data.groupby(['year', group_variable])['SCORE'].agg(pd.Series.mode)
        
        # get average of mode values where two have been returned
        true_mode = []
        for x in self.return_data['mode']:
            try: 
                mode = sum(x)/len(x)
                true_mode.append(mode)
            except:
                true_mode.append(x)
        self.return_data['mode'] = true_mode
        
        self.return_data.rename(columns={'SCORE': 'mean'}, inplace = True)
        self.return_data.sort_values('mean', inplace=True)
        return self.return_data


    

In [7]:
class ZipCorrelationData:
    def __init__(self):
        self.zipcode_violations = None
        self.graph_data = None
        self.order = None
        self.aov = None

    
    def generate_base_data(self):
        # get serial number and violation codes from all violation documents in the database
        violations = pd.DataFrame(Database.DATABASE['FACILITIES']\
                    .find({'VIOLATION CODE': {'$exists': True}}, {'_id': 0, 'SERIAL NUMBER': 1, 'VIOLATION CODE': 1}))
        
        # group by serial number and count, provides that total number of violations per inspection
        serialno_violations = violations.groupby('SERIAL NUMBER').count()
        
        
        # get facility id and serial number from all inspection documents in the database.
        facilities_serialnos = pd.DataFrame(Database.DATABASE['FACILITIES'].find({'ACTIVITY DATE':{'$exists': True}},\
                                                    {'_id': 0, 'FACILITY ID': 1, 'SERIAL NUMBER': 1, 'Zip Codes': 1}))
        
        # merge and groupby facilities to provide total number of violations by facility 
        self.zipcode_violations = pd.merge(serialno_violations, facilities_serialnos, on = 'SERIAL NUMBER').\
                                                    groupby('FACILITY ID').sum()
        
        # final cleaning: remove inactive, remove na and equivalent missing values 'na' and 0 zip codes
        self.zipcode_violations = Cleaning().remove_inactive(self.zipcode_violations)
        self.zipcode_violations.dropna(0, inplace = True)
        self.zipcode_violations.drop(self.zipcode_violations[self.zipcode_violations['Zip Codes'] == 0]\
                                     .index, inplace = True)
        self.zipcode_violations.rename({'VIOLATION CODE': 'sum_violations', 'Zip Codes': 'zip_codes'},\
                                       axis = 1, inplace=True)

        
    def get_anova(self):
        # uses pingoiun package to calculate one way anova, zip codes as independent variable.
        try:
            self.aov = pg.anova(dv = 'sum_violations', between = 'zip_codes', data = self.zipcode_violations,\
                                detailed = True, effsize = 'n2')
        except:
            self.generate_base_data()
            self.aov = pg.anova(dv = 'sum_violations', between = 'zip_codes', data = self.zipcode_violations,\
                                detailed = True, effsize = 'n2')
        return self.aov
    
    
    def get_boxplot_data(self):
        try:
            self.graph_data = self.zipcode_violations.groupby('zip_codes').sum()
        except:
            self.generate_base_data()
            self.graph_data = self.zipcode_violations.groupby('zip_codes').sum()
        
        self.graph_data.reset_index(inplace=True)
        self.graph_data['zip_abb'] = self.graph_data['zip_codes'].apply(lambda x: int(str(x)[0:2]))
        self.graph_data = self.graph_data.groupby('zip_abb')['sum_violations'].apply(np.array)
        self.means = pd.Series([sum(x)/len(x) for x in self.graph_data], index = self.graph_data.index, name='means')
        self.sorter = pd.merge(self.graph_data, self.means, on = 'zip_abb')
        self.order = self.sorter.sort_values('means', ascending = False).index
        
        return self.graph_data, self.order


# TKINTER

In [8]:
class RestDataApp(tk.Tk):
    #root class, homeframe created using composition
    def __init__(self):
        tk.Tk.__init__(self)
        self.geometry('1000x500')
        self.title('Restaurant Data Application')
        self.homescreen = HomeScreen(self)
        self.configure(bg = 'gray19')
        self.columnconfigure(0, weight = 1)
        self.rowconfigure(0, weight = 1)
        self.homescreen.grid(column = 0, row = 0, sticky = 'nsew')

In [10]:
class HomeScreen(tk.Frame):
    def __init__(self, master):
        # override master exception reporting with bespoke class to capture application specific exceptions
        master.report_callback_exception = self.report_callback_exception
        self.master = master
        tk.Frame.__init__(self, master, bg = 'gray19')
        self.columnconfigure((0,1), weight = 1)
        self.rowconfigure(0, weight = 1)
        
        # dashboard to show database status on homescreen. Ideally graph for optimum user experience
        self.dashboard = DashBoard(self)
        self.dashboard.grid(column = 1, row = 0, sticky = 'nsew', padx = 40, pady = 10)
        
        # composition
        self.widgets = HomePageWidgets(self)
        self.widgets.grid(column = 0, row = 0, sticky = 'nsew', padx = 40, pady = 10)
        
        self.quit_button(master)
        self.report_callback_exception
    
    def quit_button(self, master):
        self.quit_button = OptionButton(self, text = 'Exit the application', command = master.destroy)
        self.quit_button.grid(column = 1, row = 1, padx = 40, pady = 10, sticky = 'e')

        
        # calls to 3 sub-screens follow, corresponding to the three core application commands shown as buttons on the
        ## home screen. All layer on top of the home screen
    def upload(self):
        self.upload_frame = UploadScreen(self)
        self.upload_frame.grid(column = 0, row = 0, columnspan = 3, rowspan = 3, sticky = 'NSEW', padx = 40, pady = 10)
        
    def clean(self):
        self.clean_frame = CleanScreen(self, self.master)
        self.clean_frame.grid(column = 0, row = 0, columnspan = 3, rowspan = 3, sticky = 'NSEW', padx = 40, pady = 10)
        
    def analyse(self):
        self.analyse_frame = AnalysisScreen(self, self.master)
        self.analyse_frame.grid(column =0, row = 0, columnspan = 3, rowspan = 3, sticky = 'NSEW', padx = 40, pady = 10)

        
        # bespoke error reporting using traceback and messagebox. Known errors have bespoke handling, otherwise
        ## traceback is show in messagebox
    def report_callback_exception(self, *args):
        err = traceback.format_exception(*args)
        if 'KeyError' in str(err):
            messagebox.showerror('Error', 'Unable to complete the operation as insufficient data was found. Please ensure you have uploaded all required files and completed data cleaning actions')
        else:
            messagebox.showerror('Error', f'Sorry, an unknown error occured. \n {str(err)}')

    def quit_frame(self, frame):
        # update dashboard when leaving one of the command frames
        self.dashboard.refresh_dashboard()
        frame.grid_forget()

In [11]:
class HomePageWidgets(tk.Frame):
    
    # 3 central command buttons of the application

    def __init__(self, master):
        tk.Frame.__init__(self, master, borderwidth = 2, bg = 'gray77')
        self.columnconfigure(0, weight = 1)
        self.rowconfigure((1,2,3), weight = 1)
           
        # images and triadic colours used for 3 core command buttons
        self.upload_pic = ImageTk.PhotoImage(Image.open('upload.png'))
        self.upload_button = tk.Button(self, text = 'Upload new data', command = master.upload, image = self.upload_pic, compound = 'top', bg = 'SteelBlue1')
        self.upload_button.grid(column = 0, row = 1, padx = 40, pady= 10, sticky = 'nsew')
    
    
        self.clean_pic = ImageTk.PhotoImage(Image.open('clean.png'))
        self.clean_button = tk.Button(self, text = 'Clean my data', command = master.clean, image = self.clean_pic, compound = 'top', bg = 'DarkOrange1').\
                grid(column = 0, row = 2, padx = 40, pady= 10, sticky = 'nsew')
        
        
        self.analyse_pic = ImageTk.PhotoImage(Image.open('statistical-analysis.png'))
        self.analyse_button = OptionButton(self, text = 'Analyse my data', command = master.analyse, image = self.analyse_pic, compound = 'top', bg = 'green2').\
            grid(column = 0, row = 3, padx = 40, pady= 10, sticky = 'nsew')
        
        
        Label(self, text = 'TAKE ACTION', font = ('Helevtica', 16, 'bold'), justify = 'center', bg = 'gray77').grid(row = 0, column = 0)

In [12]:
class DashBoard(tk.Frame):
    def __init__(self, master):
        
        # dashboard used to show database status on the home screen
        tk.Frame.__init__(self, master, bg = 'gray77', relief = 'sunken', borderwidth = 2)
        self.master = master
        self.columnconfigure(0, weight = 1)
        self.rowconfigure((0,1,2,3,4,5), weight = 1)
        self.refresh_dashboard()
    
    def refresh_dashboard(self):
        self.inv_value = Database.count("FACILITIES", "Inventory")
        self.ins_value = Database.count("FACILITIES", "Inspections")
        self.vio_value = Database.count("FACILITIES", "Violations")
        self.seats_value = Database.count("FACILITIES", "Seats")
        self.prog_label = Database.count("FACILITIES", "Program Status")
        
        self.title = Label(self, textvariable = Text('Database status'), font = ('helvetica', 18, 'bold'), justify = 'center').grid(row = 0, sticky = 'nsew')
        
        self.inv_label = Label(self, textvariable = Text(f'Inventory Records: \t\t\t {self.inv_value}'))\
            .grid(column = 0, row = 1, sticky = 'w')
        self.ins_label = Label(self, textvariable = Text(f'Inspection Records: \t\t\t {self.ins_value}'))\
              .grid(column = 0, row = 2, sticky = 'w')
        self.vio_label = Label(self, textvariable = Text(f'Violation Records: \t\t\t {self.vio_value}'))\
            .grid(column = 0, row = 3, sticky = 'w')
        self.seats_label = Label(self, textvariable = Text(f'Facility seats records: \t\t {self.seats_value}') )\
            .grid(column = 0, row = 4, sticky = 'w')
        self.prog_label = Label(self, textvariable = Text(f'Facility program status records: \t {self.prog_label}'))\
            .grid(column = 0, row = 5, sticky = 'w')
        self.update_idletasks()
        

## Bespoke tkinter widget classes, created for code reuse and control of themes/colours

In [13]:
class QuitButton(tk.Button):
    def __init__(self, master, root, text, command, bg = None):
        tk.Button.__init__(self, master)
        self.config(text = text, command = command, bg = bg)

In [14]:
class OptionButton(tk.Button):
    def __init__(self, master = None, command = None, text = None, textvariable = None, image = None, compound = None, bg = None, wraplength = None):
        tk.Button.__init__(self, master)
        self.config(text = text, textvariable = textvariable, command = command, image = image, compound = compound, bg = bg, wraplength = wraplength)

In [15]:
class Frame(tk.Frame):
    def __init__(self, master, bg = 'gray77', borderwidth = 2, relief = 'sunken', image = None, compound = None):
        tk.Frame.__init__(self, master)
        self.config(bg = bg, borderwidth = borderwidth, relief = relief, image = image, compound = compound)

In [16]:
class Label(tk.Label):
    def __init__(self, master, text = None, textvariable = None, font = ('helvetica', 12), wraplength = None, justify = None, width = None, bg = 'gray77'):
        tk.Label.__init__(self, master)
        self.config(text = text, textvariable = textvariable, bg = bg, font = font, wraplength = wraplength, justify = justify, width = width)

In [17]:
class Text(tk.StringVar):
    def __init__(self, text):
        tk.StringVar.__init__(self)
        self.set(text)

In [18]:
class Ax:
    def __init__(self, frame, figsize, title, fontsize = '25', row = None, column = None, rowspan = None, columnspan = None):
        self.fig = plt.Figure(figsize = figsize)
        self.ax = self.fig.add_subplot(111)
        self.canvas = FigureCanvasTkAgg(self.fig, frame)
        self.canvas.get_tk_widget().configure(bg = 'gray77')
        self.canvas.get_tk_widget().grid(row = row, column = column, padx = 30, pady = 30,\
                                         sticky = 'nsew', columnspan = columnspan, rowspan = rowspan)
        self.fontdict = {'fontsize': fontsize, 'fontweight': 'bold', 'verticalalignment': 'top'}
        self.ax.set_title(title, fontdict = self.fontdict)
        self.ax.set_facecolor('white')

    
    def get(self):
        return self.fig, self.ax

## 3 screens corresponding to the central commands of the application

In [19]:

class UploadScreen(Frame):
    def __init__(self, master):
        Frame.__init__(self, master)
        
        self.columnconfigure(0, weight = 1, minsize = 300)
        self.rowconfigure((0,1,2,3,4), weight = 1)
        
        # image and colour consistent with home screen
        self.upload_pic = ImageTk.PhotoImage(Image.open('upload.png'))
        self.img_header_label = tk.Label(self, text = 'Upload new data', image = self.upload_pic,
                                         compound = 'top', bg = 'SteelBlue1', borderwidth = 2)
        self.img_header_label.grid(column = 0, row = 0, sticky = 'nw')
        
        self.choice_text = Text('Click here to choose a file to upload')
        self.choice_button = OptionButton(self, textvariable = self.choice_text, command = self.get_url, bg = 'SteelBlue1')\
            .grid(column = 0, row = 1, padx = 100, pady=40, sticky = 'nsew')
        
    
        QuitButton(self, master, 'Back', command = lambda: master.quit_frame(self))\
            .grid(column = 0, row = 5, padx = 20, pady=20, sticky = 'e')
        
        self.url = Text('')
        Label(self, textvariable = self.url, wraplength = 500, justify = 'center')\
            .grid(column = 0, row = 2, sticky = 'ew')
        
        self.uploading_status = Text('')
        Label(self, textvariable = self.uploading_status)\
            .grid(column = 0, row = 4)
        
        self.completion_conf = Text('')
        Label(self, textvariable = self.completion_conf)\
            .grid(column = 0, row = 4)
        
               
            
    def get_url(self):
        self.url.set(filedialog.askopenfilename())
        if len(self.url.get()) != 0:
            self.choice_text.set('Make another selection?')
            self.confirm_selection()
        else:
            self.uploading_status.set('No file selected')
            self.update_idletasks()
        
            
    def confirm_selection(self):
        self.confirm_button = OptionButton(self, text = 'Confirm upload of file to the database?', command = self.uploader, bg = 'SteelBlue4')
        self.confirm_button.grid(column = 0, row = 3)
        
    def uploader(self):
        upload_url = self.url.get()
        self.uploading_status.set('Upload in progress...')
        self.update_idletasks()
        Importer().generic_importer(upload_url)
        self.confirm_button.grid_forget()
        self.uploading_status.set('Upload complete!')
        self.update_idletasks()
      

In [20]:

class CleanScreen(tk.Frame):
    def __init__(self, master, root):
        root.report_callback_exception = self.report_callback_exception
        tk.Frame.__init__(self, master, bg = 'gray77', relief = 'sunken', borderwidth = 2)

        self.columnconfigure((0,1,2), weight = 1, minsize = 150)
        self.rowconfigure((1,2), weight = 1)
        self.rowconfigure(0, weight = 1, minsize = 250)
        self.rowconfigure(2, minsize = 150)
   
        #image and colour consistent with parent button on homescreen
        self.upload_pic = ImageTk.PhotoImage(Image.open('clean.png'))
        self.img_header_label = tk.Label(self, text = 'Clean my data', image = self.upload_pic,
                                         compound = 'top', bg = 'DarkOrange1', borderwidth = 2)
        self.img_header_label.grid(column = 0, row = 0, sticky = 'nw')

        
        OptionButton(self, text = 'Remove duplicates', command = self.choose_duplicates, bg = 'DarkOrange1')\
            .grid(column = 0, row = 0, padx = 40, pady = 100, sticky = 'nsew')
        OptionButton(self, text = 'Extract seating data', command = self.get_seating, bg = 'DarkOrange1')\
            .grid(column = 1, row = 0, padx = 40, pady = 100, sticky = 'nsew')
        OptionButton(self, text = 'Extract program status', command = self.get_program_status, bg = 'DarkOrange1')\
            .grid(column = 2, row = 0,  padx = 40, pady = 100, sticky = 'nsew')
        QuitButton(self, master, 'Back', command = lambda: master.quit_frame(self))\
            .grid(column = 3, row = 2, padx = 20, pady = 20, sticky = "e")
        
        # subframe used to house actions in the following methods
        self.subframe = None

    
    def choose_duplicates(self):      
        self.subframe = Frame(self, relief = None, borderwidth = None, bg = 'gray77')
        self.subframe.grid(row = 1, column = 0, rowspan = 2, columnspan = 3)
        
        self.dup_label = Label(self.subframe, text = 'Choose record to delete duplicates from:', bg = 'gray77')
        self.dup_label.grid(column = 0, row = 0)
        
        self.documents = ['Inventory', 'Inspections', 'Violations']
        self.var = Text(self.documents[0])
        self.doc_menu = tk.OptionMenu(self.subframe, self.var, *self.documents)
        self.doc_menu.config(bg = 'DarkOrange3')
        self.doc_menu.grid(column = 1, row = 0, sticky = 'nsew', padx = 40, pady = 40)
        self.var.trace('w', self.erase_duplicates)
        

    
    def erase_duplicates(self, *args):
        self.doc_menu.grid_forget()
        self.dup_label.grid_forget()
        self.subframe.update_idletasks()
        
        self.status = Text('')
        self.update_label = Label(self.subframe, textvariable = self.status, wraplength = 200, width = 20, justify = 'center')\
            .grid(column = 0, row = 2, columnspan = 2, padx = 40, pady = 20, sticky = 's')
        
        self.doc_choice = self.var.get()
        self.status.set(f'Erasing {self.doc_choice} records')
        self.update_idletasks()
        
        self.identity_field = Database.identity_dict(self.doc_choice)
        self.type_field = Database.type_dict(self.doc_choice)       
        
        self.deleted_count = Cleaning().delete_duplicates(self.type_field, self.identity_field)
        self.status.set(f'Deleted duplicate {self.doc_choice} records: {self.deleted_count}')
        self.update_idletasks()

        self.after(5000, self.status.set(''), self.subframe.grid_forget)
        self.update_idletasks()
    
    
    def get_seating(self):
        self.subframe = Frame(self, relief = None, borderwidth = None)
        self.subframe.grid(row = 1, column = 0, rowspan = 3, columnspan = 3)
        self.status = Text('')
        self.update_label = Label(self.subframe, textvariable = self.status, wraplength = 200, width = 20, justify = 'center')\
            .grid(column = 0, row = 2, padx = 40, pady = 20, sticky = 'nsew')
        self.status.set('Creating seating data...')
        self.update_idletasks()
        
        self.seats = Cleaning().get_seating_info()
        self.status.set(self.seats)
        self.update_idletasks()
        
        self.after(5000, self.status.set(''))
        self.update_idletasks()
        
    def get_program_status(self):
        self.subframe = Frame(self, relief = None, borderwidth = None)
        self.subframe.grid(row = 1, column = 0, rowspan = 3, columnspan = 3)
        self.status = Text('')
        self.update_label = Label(self.subframe, textvariable = self.status, wraplength = 200, width = 20,\
                                  justify = 'center')\
            .grid(column = 0, row = 2, padx = 40, pady = 20, sticky = 'nsew')
        self.status.set('Creating program data...')
        self.update_idletasks()
        
        self.status.set(Cleaning().insert_prog_status())
        self.update_idletasks()
        
        self.after(5000, self.status.set(''))
        self.update_idletasks()
        
    def report_callback_exception(self, *args):
        err = traceback.format_exception(*args)
        if 'command document too large' in str(err):
            messagebox.showerror('Error', 'Database unable to process this quantity of duplicates')
        elif 'KeyError' in str(err):
            messagebox.showerror('Error', 'Unable to complete the operation as insufficient data was found. Please ensure you have uploaded all required files and completed data cleaning actions')
        else:
            messagebox.showerror('Error', f'Sorry, an unknown error occured. \n {str(err)}' )
        self.status.set('')
        self.update_idletasks()

In [21]:
class AnalysisScreen(tk.Frame):
    def __init__(self, master, root):
        tk.Frame.__init__(self, master, bg = 'gray77', relief = 'sunken', borderwidth = 2)
        self.columnconfigure((0,1,2), weight = 1, minsize = 150)
        self.rowconfigure((1,2), weight = 1)
        self.rowconfigure(0, weight = 1, minsize = 250)
        self.rowconfigure(2, minsize = 150)
        
        # image and colour consistent with parent button on homescreen
        self.upload_pic = ImageTk.PhotoImage(Image.open('statistical-analysis.png'))
        self.img_header_label = tk.Label(self, text = 'Analyse my data', image = self.upload_pic,
                                         compound = 'top', bg = 'green2', borderwidth = 2)
        self.img_header_label.grid(column = 0, row = 0, sticky = 'nw')
        
        OptionButton(self, text = 'Facility count by violation', command = self.violations_graph, wraplength = 90, bg = 'green3')\
                .grid(column = 0, row = 0,  padx = 40, pady = 100, sticky = 'nsew')
        OptionButton(self, text = 'Measures of central tendency', command = self.mean_med_mode, wraplength = 90, bg = 'green3')\
                .grid(column = 1, row = 0,  padx = 40, pady = 100, sticky = 'nsew')
        OptionButton(self, text = 'Correlation', command = self.zip_correlation, wraplength = 90, bg = 'green3')\
                .grid(column = 2, row = 0,  padx = 40, pady = 100, sticky = 'nsew')
        OptionButton(self, text = 'Back', command = lambda: master.quit_frame(self), wraplength = 90)\
            .grid(column = 3, row = 3, padx = 20, pady = 10, sticky = "e")

    def violations_graph(self):
        self.vio_frame = ViolationsGraph(self)
    
    
    def mean_med_mode(self):
        self.mean_med_mode = MeanMedModeGraph(self)
        
    def zip_correlation(self):
        self.zip_corr = ZipCorrelationScreen(self)

        
    

## Tk Toplevels used for data visualisations

In [22]:
class ViolationsGraph(tk.Toplevel):
    def __init__(self, master):
        tk.Toplevel.__init__(self, master, bg = 'white')
        self.update_label = Label(self, text = 'Working on it...', bg = 'white')
        self.update_label.grid()
        self.graph()
        
        
    def graph(self):
        self.data = ViolationsData().get_violations_graph_data()
        self.update_label.grid_forget()
        self.fig, self.ax = Ax(self, (12,5), 'Sum of restaurants that have commited each violation', row = 0, column = 0).get()
        for tick in self.ax.get_xticklabels():
            tick.set_rotation(45)
            tick.set_fontsize(8)
        self.data.plot(kind = 'bar', ax = self.ax)
        self.ax.set_xlabel('Violation Code', fontsize = 15)
        self.ax.set_ylabel('Number of restaurants', fontsize = 15)
        
        

In [23]:
class MeanMedModeGraph(tk.Toplevel):
    def __init__(self, master):
        tk.Toplevel.__init__(self, master, bg = 'white')
        self.columnconfigure(0, weight = 1)
        self.rowconfigure((0,1), weight = 1)
        self.master = master
        self.options = ['Zip Codes', 'seats']
        self.var = tk.StringVar()
        self.var.set(self.options[0])
        
        self.opt_label = Label(self, text = 'Select attribute to see results grouped by:', bg = 'white')
        self.opt_label.grid(row = 0, column = 0, padx = 10)
        
        self.group_variable_option = tk.OptionMenu(self, self.var, *self.options)
        self.group_variable_option.config(bg = 'green4')
        self.group_variable_option.grid(row = 0, column = 1, sticky = 'w', padx = 50)
        self.var.trace('w', self.graph)
        
        self.var.trace('w', self.graph)
        
    def graph(self, *args):
        self.subframe = Frame(self, relief = None, borderwidth = 2, bg = 'white')
        self.subframe.grid(row = 1, column = 0, rowspan = 3, columnspan = 3, sticky = 'nsew')
        self.subframe.rowconfigure((0,1,2,3), weight = 1)
        self.subframe.columnconfigure(0, weight = 1)
        self.update_label = Label(self.subframe, 'Working on it...', bg = 'white')
        self.update_label.grid(row = 0, column = 0)
        self.update_idletasks()
        
        self.group_variable = self.var.get()
        self.data = MeanMedModeData().wrangle_mean_med_data(self.group_variable)
        self.update_label.grid_forget()
        
        self.header = Label(self.subframe, text = f'Mean, Median and Mode inspection score by {self.group_variable}',\
                            bg = 'white')
        self.header.config(font = '40')
        self.header.grid(row = 0, column =0)
        
        self.fig1, self.ax1 = Ax(self.subframe, (12,2.5), '2017', row = 1, column = 0).get()
        self.ax1.set_ylim(75, 100)
        self.data.loc[2017].plot(kind = 'bar', ax = self.ax1)

        self.fig2, self.ax2 = Ax(self.subframe, (12,2.5), '2018', row = 2, column = 0).get()
        self.ax2.set_ylim(75, 100)
        self.data.loc[2018].plot(kind = 'bar', ax = self.ax2)

        self.fig3, self.ax3 = Ax(self.subframe, (12,2.5), '2019', row = 3, column = 0).get()
        self.ax3.set_ylim(75, 100)
        self.data.loc[2019].plot(kind = 'bar', ax = self.ax3)
        
        Label(self.subframe, text = f'{self.group_variable}')
        Label(self, text = 'Measure')

In [27]:
class ZipCorrelationScreen(tk.Toplevel):
    def __init__(self, master):
        tk.Toplevel.__init__(self, master, bg = 'white')
        self.columnconfigure(0, weight = 1)
        self.rowconfigure(0, weight = 1)
        self.master = master
        self.update_label = Label(self, text = 'Working on it...')
        self.update_label.grid(row = 0, column = 0)
        self.update_idletasks()
        self.graph()
        self.anova_values()

        
    def graph(self):
        self.update_label.grid_forget()
        self.graph_data, self.order = ZipCorrelationData().get_boxplot_data()
        self.fig1, self.ax1 = Ax(self, (12,7), 'Mean and variance by two digit zip code',\
                                 row = 0, column = 0, rowspan = 3).get()
        
        for tick in self.ax1.get_xticklabels():
            tick.set_rotation(90)
            tick.set_fontsize(8)
        
        
        x = 0
        for idx in self.order:
            self.ax1.boxplot(self.graph_data[idx], positions = [x], patch_artist = True,\
                        showfliers = False, widths = 0.65, meanline = True, showmeans = True,\
                        boxprops=dict(facecolor= 'lightcoral'),\
                        meanprops=dict(color= 'black', linewidth = 2, linestyle = '-' ))                 
            x +=1
        self.ax1.set_xlabel('Zip codes', fontsize = 15)
        self.ax1.set_ylabel('Number of violations', fontsize = 15)
        
    def anova_values(self):
        self.aov_df = ZipCorrelationData().get_anova()
        self.aov_subframe = Frame(self)
        Label(self.aov_subframe, text = "One Way Anova results", font = ('helvetica', 15, 'bold'))\
                .grid(row = 0, sticky = 'ew')
        Label(self.aov_subframe, text = f'N2:\t{self.aov_df.loc[0]["n2"].round(2)}',\
              font = ('helvetica', 15)).grid(row = 1, padx = 10, pady = 10, sticky = 'ew')
        Label(self.aov_subframe, text = f'P Value:\t{self.aov_df.loc[0]["p-unc"].round(2)}',\
              font = ('helvetica', 15)).grid(row = 2, padx = 10, pady = 10, sticky = 'ew')
        self.aov_subframe.grid(row = 3, column = 0, padx = 10, pady = 10, sticky = 'ew')
        

# Main method to run the application

In [28]:
if __name__ == "__main__":
    Database.initialise()
    app = RestDataApp()
    app.mainloop()